 ## Step2：微调

### 1. 环境准备

In [1]:
# 查看已安装依赖
! pip list

Package                           Version
--------------------------------- --------------------
absl-py                           2.1.0
accelerate                        0.33.0
adaseq                            0.6.6
addict                            2.4.0
aiohttp                           3.9.5
aiosignal                         1.3.1
albucore                          0.0.12
albumentations                    1.4.11
alias-free-torch                  0.0.6
aliyun-python-sdk-core            2.15.1
aliyun-python-sdk-kms             2.16.3
altair                            5.4.0
aniso8601                         9.0.1
annotated-types                   0.7.0
antlr4-python3-runtime            4.9.3
anyio                             4.4.0
apex                              0.1
appdirs                           1.4.4
argon2-cffi                       23.1.0
argon2-cffi-bindings              21.2.0
arrow                             1.3.0
asttokens                         2.4.1
astunparse        

In [2]:
# 安装 streamlit
! pip install streamlit==1.24.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


### 2.2 模型下载

In [3]:
from modelscope import snapshot_download
model_dir = snapshot_download('IEITYuan/Yuan2-2B-Mars-hf', cache_dir='.')

Downloading: 100%|██████████| 0.98k/0.98k [00:00<00:00, 1.91kB/s]
Downloading: 100%|██████████| 0.98k/0.98k [00:00<00:00, 1.80kB/s]
Downloading: 100%|██████████| 51.0/51.0 [00:00<00:00, 105B/s]
Downloading: 100%|██████████| 1.29k/1.29k [00:00<00:00, 2.61kB/s]
Downloading: 100%|██████████| 144/144 [00:00<00:00, 259B/s]
Downloading: 100%|██████████| 7.86k/7.86k [00:00<00:00, 15.6kB/s]
Downloading: 100%|██████████| 4.41G/4.41G [00:16<00:00, 288MB/s] 
Downloading: 100%|██████████| 7.61k/7.61k [00:00<00:00, 14.3kB/s]
Downloading: 100%|██████████| 411/411 [00:00<00:00, 938B/s]
Downloading: 100%|██████████| 2.06M/2.06M [00:00<00:00, 4.17MB/s]
Downloading: 100%|██████████| 1.12k/1.12k [00:00<00:00, 2.60kB/s]
Downloading: 100%|██████████| 52.0k/52.0k [00:00<00:00, 95.6kB/s]
Downloading: 100%|██████████| 52.0k/52.0k [00:00<00:00, 97.1kB/s]


### 2.3 数据处理

当前安装的Keras版本（Keras 3）与Transformers库不兼容。Transformers库尚未支持Keras 3。为了解决这个问题，您需要安装与TensorFlow兼容的旧版Keras，即tf-keras。

In [4]:
!pip uninstall keras -y
!pip install tf-keras

Found existing installation: keras 3.5.0
Uninstalling keras-3.5.0:
  Successfully uninstalled keras-3.5.0
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 294.1 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
# 导入环境
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

2024-08-14 10:12:57.578583: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 10:12:57.589979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 10:12:57.603486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 10:12:57.607541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 10:12:57.617774: I tensorflow/core/platform/cpu_feature_guar

In [6]:
# 读取数据
df = pd.read_json('./data.json')
ds = Dataset.from_pandas(df)

In [7]:
# 查看数据
len(ds)
ds[:1]

{'input': ['# 角色\n你是一个资深的 AI 翻译助理，具备卓越的语言能力，能够精准、流畅且地道地翻译英语文本。\n\n## 技能\n### 技能 1: 各类文本翻译\n1. 当用户提供需要翻译的文本时，深入准确地理解其含义、语境和文化背景。\n2. 充分运用丰富全面的语言知识、翻译技巧以及专业工具，将其高质量地翻译成中文。回复示例：\n=====\n\n    n-  原文: <用户提供的原文>\n   -  译文: <翻译后的文本>\n=====\n\n### 技能 2: 处理专业术语\n1. 对于包含专业术语的文本，通过专业词典和权威资料进行准确翻译。\n2. 必要时为用户解释专业术语在特定领域的含义。回复示例：\n=====\n   -  原文: <用户提供的原文>\n   -  译文: <翻译后的文本>\n   -  术语解释: <对文中专业术语的解释>\n=====\n\n## 限制:\n- 仅专注于翻译相关的任务，坚决拒绝回答与翻译无关的任何问题。\n- 所输出的内容务必严格按照给定的格式进行组织，不得偏离框架要求。\n- 译文要清晰易懂、自然流畅，避免使用生僻或过于复杂的中文表述。\n\nAMP-activated protein kinase (AMPK) is a regulator for energy metabolism in cells and plays a key role in the energy balance.'],
 'output': ['腺苷酸活化蛋白激酶(AMP-activated protein kinase，AMPK)是细胞的能量代谢调节器，在能量平衡中起关键作用。']}

In [8]:
# 加载 tokenizer
path = './IEITYuan/Yuan2-2B-Mars-hf'

tokenizer = AutoTokenizer.from_pretrained(path, add_eos_token=False, add_bos_token=False, eos_token='<eod>')
tokenizer.add_tokens(['<sep>', '<pad>', '<mask>', '<predict>', '<FIM_SUFFIX>', '<FIM_PREFIX>', '<FIM_MIDDLE>','<commit_before>','<commit_msg>','<commit_after>','<jupyter_start>','<jupyter_text>','<jupyter_code>','<jupyter_output>','<empty_output>'], special_tokens=True)
tokenizer.pad_token = tokenizer.eos_token

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

In [9]:
# 定义数据处理函数
def process_func(example):
    MAX_LENGTH = 512    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性

    instruction = tokenizer(f"{example['input']}<sep>")
    response = tokenizer(f"{example['output']}<eod>")
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = [1] * len(input_ids) 
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] # instruction 不计算loss

    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [10]:
# 处理数据集
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/540 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 540
})

In [11]:
# 数据检查
tokenizer.decode(tokenized_id[0]['input_ids'])

'# 角色\n你是一个资深的 AI 翻译助理，具备卓越的语言能力，能够精准、流畅且地道地翻译英语文本。\n\n## 技能\n### 技能 1: 各类文本翻译\n1. 当用户提供需要翻译的文本时，深入准确地理解其含义、语境和文化背景。\n2. 充分运用丰富全面的语言知识、翻译技巧以及专业工具，将其高质量地翻译成中文。回复示例：\n=====\n\n    n-  原文: <用户提供的原文>\n   -  译文: <翻译后的文本>\n=====\n\n### 技能 2: 处理专业术语\n1. 对于包含专业术语的文本，通过专业词典和权威资料进行准确翻译。\n2. 必要时为用户解释专业术语在特定领域的含义。回复示例：\n=====\n   -  原文: <用户提供的原文>\n   -  译文: <翻译后的文本>\n   -  术语解释: <对文中专业术语的解释>\n=====\n\n## 限制:\n- 仅专注于翻译相关的任务，坚决拒绝回答与翻译无关的任何问题。\n- 所输出的内容务必严格按照给定的格式进行组织，不得偏离框架要求。\n- 译文要清晰易懂、自然流畅，避免使用生僻或过于复杂的中文表述。\n\nAMP-activated protein kinase (AMPK) is a regulator for energy metabolism in cells and plays a key role in the energy balance.<sep> 腺苷酸活化蛋白激酶(AMP-activated protein kinase，AMPK)是细胞的能量代谢调节器，在能量平衡中起关键作用。<eod>'

In [12]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[0]["labels"])))

'腺苷酸活化蛋白激酶(AMP-activated protein kinase，AMPK)是细胞的能量代谢调节器，在能量平衡中起关键作用。<eod>'

### 2.4 模型训练

In [13]:
# 模型加载
model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model

YuanForCausalLM(
  (model): YuanModel(
    (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
    (layers): ModuleList(
      (0-23): 24 x YuanDecoderLayer(
        (self_attn): YuanAttention(
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): YuanRotaryEmbedding()
          (lf_gate): LocalizedFiltering(
            (conv1): Conv2d(2048, 1024, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (conv2): Conv2d(1024, 2048, kernel_size=(2, 1), stride=(1, 1), padding=(1, 0))
            (output_layernorm): YuanRMSNorm()
          )
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): YuanMLP(
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (gate_proj): Linear(in_features=2048, out_featu

In [14]:
model.enable_input_require_grads() # 开启gradient_checkpointing时，要执行该方法

In [15]:
# 查看模型数据类型
model.dtype

torch.bfloat16

In [16]:
# 配置Lora
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj', 'gate_proj', 'up_proj', 'o_proj', 'k_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [17]:
# 构建PeftModel
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): YuanForCausalLM(
      (model): YuanModel(
        (embed_tokens): Embedding(135040, 2048, padding_idx=77185)
        (layers): ModuleList(
          (0-23): 24 x YuanDecoderLayer(
            (self_attn): YuanAttention(
              (v_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (o_proj): lora.Linear(
                (base_layer): Linear(in_

In [18]:
# 打印需要训练的参数
model.print_trainable_parameters()

trainable params: 9,043,968 || all params: 2,097,768,448 || trainable%: 0.4311


In [27]:
# 设置训练参数
args = TrainingArguments(
    output_dir="./output/Yuan2.0-2B_lora_bf16",
    per_device_train_batch_size=12,
    gradient_accumulation_steps=1,
    logging_steps=1,
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    bf16=True
)

In [28]:
# 初始化Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
# 模型训练
trainer.train()

Step,Training Loss
1,1.899400
2,1.391700
3,1.799100
4,1.674200
5,1.937900
6,2.148900
7,1.721300
8,1.567000
9,1.886900
10,1.647600


/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./IEITYuan/Yuan2-2B-Mars-hf - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ./IEITYuan/Yuan2-2B-Mars-hf - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the

TrainOutput(global_step=135, training_loss=1.336738242926421, metrics={'train_runtime': 413.1937, 'train_samples_per_second': 3.921, 'train_steps_per_second': 0.327, 'total_flos': 8445619443990528.0, 'train_loss': 1.336738242926421, 'epoch': 3.0})

### 2.5 效果验证

In [22]:
# 定义生成函数
def generate(prompt):
    prompt = prompt + "<sep>"
    inputs = tokenizer(prompt, return_tensors="pt")["input_ids"].cuda()
    outputs = model.generate(inputs, do_sample=False, max_length=1024)
    output = tokenizer.decode(outputs[0])
    print(output.split("<sep>")[-1])

In [23]:
# 输入prompt template
template = '''
# 角色
你是一个资深的 AI 翻译助理，具备卓越的语言能力，能够精准、流畅且地道地翻译英语文本。

# 技能
1. 当用户提供英语，你输出中文

# 回复示例
输入： Previously we obtained recombinant soluble human rhIFN-λ1 from Pichia pastoris.
输出: 此前，我们已经从毕赤酵母表达获得了可溶性重组人干扰素-λ1。

# 限制:
- 仅专注于翻译相关的任务，坚决拒绝回答与翻译无关的任何问题。
- 译文要清晰易懂、自然流畅，避免使用生僻或过于复杂的中文表述。

input_str

'''


In [32]:
input_str = '"Results: Fourteen patients, including 5 males and 9 females, were (55±8) years old, there were 9 patients with single lesion and 5 patients with multiple lesions (2-6 lesions). a total of 26 lesions were identified."'
prompt = template.replace('input_str', input_str).strip()
generate(prompt)

 结果：共纳入14例患者，年龄5～9岁，其中5例男，9例女，9例单个病灶，5例多发性病灶，共计26个病灶。<eod>
